In [71]:
import requests
from bs4 import BeautifulSoup
import telebot
import time
import json

SLEEP_TIME = 2 #time in seconds to avoid timeout error using telegram api
TOKEN = "2026736293:AAHtfWtfMYlnQuPP9C0raA4Ow2Sa91VEfjU"
CHAT_ID = "-1001438784407"
request_url = "https://api.telegram.org/bot" + TOKEN

In [72]:
f = open("lastPostIdCrawled.txt", "r")
lastIdCrawled = f.read()

In [73]:
URL = "https://foros.3dgames.com.ar/threads/942062-ofertas-online-argentina"
landingPage = BeautifulSoup(requests.get(URL).content, "html.parser")
lastThreadPageNumber = landingPage.find("span", class_="first_last").find("a")["href"][44:49]

In [74]:
crawledPosts = {}
postIds = []

pageNumberToCrawl = lastThreadPageNumber

while lastIdCrawled not in postIds:
    print("Crawling thread number:" + str(pageNumberToCrawl))
    page = BeautifulSoup(requests.get(URL + "/page" + str(pageNumberToCrawl)).content, "html.parser")
    posts = page.find("ol", class_="posts").find_all("li", class_="postbitlegacy postbitim postcontainer old")
    crawledPosts[pageNumberToCrawl] = posts
    postIds =[post['id'] for post in page.find("ol", class_="posts").find_all("span", class_="fixscroll")]
    pageNumberToCrawl = int(pageNumberToCrawl)-1

Crawling thread number:13750
Crawling thread number:13749
Crawling thread number:13748
Crawling thread number:13747
Crawling thread number:13746
Crawling thread number:13745


In [83]:
blackListedPostIds = postIds[0:postIds.index(lastIdCrawled)] 
for savedPage in reversed(crawledPosts):
    posts = crawledPosts[savedPage]
    for post in posts:
        try:
            if post.find("span", class_="fixscroll")['id'] in blackListedPostIds: continue
        except:
            continue
        sendMessage = []
        author = post.find("div", class_="popupmenu memberaction").find("strong").text
        if author == "3DGames": continue #exclude Ads
        datetime = post.find("div", class_="posthead").find("span", class_="date").text
        message = post.find("div", class_="content")
        quotes = message.find_all("div", class_="bbcode_container")

        sendMessage = datetime
        sendMessage = sendMessage + " \n" + "Author: " + author

        if len(quotes) > 0:
            for quote in quotes:
                sendMessage = sendMessage + "\n" + "Quoted message from:" + quote.find("div", class_="bbcode_postedby").find("strong").text
                sendMessage = sendMessage + "\n" + '"' + quote.find("div", class_="message").text + '"'
            for quote in message("div", class_="bbcode_container"):
                quote.decompose()

        images = message.find_all("img")
        links = message.find_all("a")

        sendMessage = sendMessage + "\n" + message.text
        
        if len(links) > 0:
            for link in links:
                if isinstance(link, type(None)):
                    continue
                sendMessage = sendMessage + "\n" + "Link:" + link["href"]
                
        params = {
            "chat_id": CHAT_ID,
            "text": sendMessage
        }
        results = requests.post(request_url + "/sendMessage", params= params)

        if len(images) > 0:
            arrayOfImages = []
            for image in images:
                if isinstance(image, type(None)):
                    continue
                if 'http' in image["src"]:
                    arrayOfImages.append({"type":"photo","media":image["src"]})
            if len(arrayOfImages) > 0:
                params = {
                    "chat_id": CHAT_ID,
                    "media": json.dumps(arrayOfImages, separators=(',', ':')),
                    "reply_to_message_id" : int(results.json()['result']['message_id'])
                }
                results = requests.post(request_url + "/sendMediaGroup", params= params)
            
        time.sleep(SLEEP_TIME)


b'{"ok":true,"result":{"message_id":320,"sender_chat":{"id":-1001438784407,"title":"3dgames ofertas","type":"channel"},"chat":{"id":-1001438784407,"title":"3dgames ofertas","type":"channel"},"date":1642972139,"text":"22-01-22,\\u00a011:11 AM \\nAuthor: Aleca\\n\\n\\nEste es el mail que me lleg\\u00f3, en la letra chica dice que es para los que le llega el mail pero peleala porque es lo mismo que me ofrecieron por telefono la mina que me llam\\u00f3.\\n\\n\\n\\nSi en la web de Vomistar vas a la parte de altas vas a ver la misma oferta, en portabilidad no es as\\u00ed. Yo se la pelear\\u00eda, el otro dia por twitter me echaron flit con el telefono fijo a pesar de tener una resoluci\\u00f3n del ENACOM a favor m\\u00edo y llamando por tel\\u00e9fono a la tarde me dieron el descuento al toque una mina que estaba haciendo homeoffice (se lo escuchaba al pibe haciendo quilombo)\\n\\n\\n\\n\\nLink:https://ibb.co/YjGsD3k\\nLink:https://ibb.co/ch5F8qN","entities":[{"offset":612,"length":22,"type

TypeError: byte indices must be integers or slices, not str

In [141]:
lastPostId = crawledPosts[lastThreadPageNumber][-1].find("span", class_="fixscroll")['id']
with open('lastPostIdCrawled.txt', 'w') as f:
    f.write(lastPostId)

In [86]:
results.text

b'{"ok":true,"result":{"message_id":320,"sender_chat":{"id":-1001438784407,"title":"3dgames ofertas","type":"channel"},"chat":{"id":-1001438784407,"title":"3dgames ofertas","type":"channel"},"date":1642972139,"text":"22-01-22,\\u00a011:11 AM \\nAuthor: Aleca\\n\\n\\nEste es el mail que me lleg\\u00f3, en la letra chica dice que es para los que le llega el mail pero peleala porque es lo mismo que me ofrecieron por telefono la mina que me llam\\u00f3.\\n\\n\\n\\nSi en la web de Vomistar vas a la parte de altas vas a ver la misma oferta, en portabilidad no es as\\u00ed. Yo se la pelear\\u00eda, el otro dia por twitter me echaron flit con el telefono fijo a pesar de tener una resoluci\\u00f3n del ENACOM a favor m\\u00edo y llamando por tel\\u00e9fono a la tarde me dieron el descuento al toque una mina que estaba haciendo homeoffice (se lo escuchaba al pibe haciendo quilombo)\\n\\n\\n\\n\\nLink:https://ibb.co/YjGsD3k\\nLink:https://ibb.co/ch5F8qN","entities":[{"offset":612,"length":22,"type

In [90]:
results.json()['result']['message_id']

320